<img style="width:700px" src="conditional_normal.png">

<img style="width:300px" src="unconscious_statistician.png">

In [63]:
import numpy as np
from numpy.linalg import inv
beta = np.array([1,2,3,0])
mu   = np.array([0,0,0,0])
cov  = np.array([[1.000, 0.000, 0.000, 0.000],
                 [0.000, 1.000, 0.000, 0.000],
                 [0.000, 0.000, 1.000, 0.999],
                 [0.000, 0.000, 0.999, 1.000]])
m = 4; feats = np.arange(4)
def f(X): return(np.matmul(X,beta))

In [64]:
from numpy.random import multivariate_normal as mvn
n = 100000
X = mvn(mu,cov,n)
print("Mean")
print(X.mean(0))
print("Covariance")
print((np.transpose(X) @ X)/(n+1))

Mean
[9.26360563e-05 3.95252692e-03 1.64010442e-03 1.48407546e-03]
Covariance
[[ 9.93760799e-01 -1.56462579e-03 -2.78458112e-04 -2.54005414e-04]
 [-1.56462579e-03  9.95928693e-01  1.10053916e-03  1.01190965e-03]
 [-2.78458112e-04  1.10053916e-03  1.00258570e+00  1.00159144e+00]
 [-2.54005414e-04  1.01190965e-03  1.00159144e+00  1.00260871e+00]]


### CES (Conditional Expectation Shapley Values) and IS (Interventional Shapley Values)

The following is only valid for a linear model currently
This is because the $f(E[X|S])=E[f(X)|S]$ only for linear models

In [18]:
from itertools import chain, combinations
from math import factorial as fact

def CES(x, DEBUG=False):
    """ Conditional Expectations Shapley

    Assuming the function is a linear model.
    
    Assuming a known covariance from a multivariate normal distribution.
    """

    def cond_dist(x2, x):
        # Return the mean and covariance of the 
        # conditional distribution: x1 | x2 = a
        # Assuming X is multivariate normal
        a   = x[x2]
        x1  = np.setdiff1d(feats, x2)
        mu1 = mu[x1]; mu2 = mu[x2]
        cov11 = cov[x1,:][:,x1]; cov12 = cov[x1,:][:,x2]
        cov21 = cov[x2,:][:,x1]; cov22 = cov[x2,:][:,x2]
        mu_new  = mu1 + cov12 @ inv(cov22) @ (a - mu2)
        cov_new = cov11 - cov12 @ inv(cov22) @ cov21
        return(mu_new, cov_new)

    def powerset(iterable):
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

    # Calculate the attributions by going over all sets
    W_sum = 0
    phi = np.zeros(4)
    for S in powerset(np.arange(4)):
        if DEBUG: print(S)

        # Create a hybrid sample 
        x_hybrid = np.copy(x)
        mu_new,_ = cond_dist(list(S), x)
        x_hybrid[np.setdiff1d(feats, list(S))] = mu_new

        # Get the weighting terms
        if len(S) != m: W_neg = fact(len(S))*fact(m-len(S)-1)/fact(m)
        if len(S) != 0: W_pos = fact(len(S)-1)*fact(m-(len(S)-1)-1)/fact(m)

        # Add the appropriate weights for each phi
        for i in feats:
            if i in S:
                phi[i] += W_pos * f(x_hybrid)
            else:
                phi[i] -= W_neg * f(x_hybrid)
    return(phi)

def IS(x, DEBUG=False):
    """
    Interventional Conditional Expectations Shapley
    
    Assuming the function is a linear model.
    """

    phi = beta * (x - mu)
    return(phi)

In [9]:
x = np.array([1.0,1.0,1.0,1.0])
print("Conditional Expectation Explanations", CES(x))
print("Interventional Explanations         ", IS(x))

Conditional Expectation Explanations [1.     2.     1.5015 1.4985]
Interventional Explanations          [1. 2. 3. 0.]


In [22]:
def cond_dist(x2, x):
    # Return the mean and covariance of the 
    # conditional distribution: x1 | x2 = a
    # Assuming X is multivariate normal
    a   = x[x2]
    x1  = np.setdiff1d(feats, x2)
    mu1 = mu[x1]; mu2 = mu[x2]
    cov11 = cov[x1,:][:,x1]; cov12 = cov[x1,:][:,x2]
    cov21 = cov[x2,:][:,x1]; cov22 = cov[x2,:][:,x2]
    mu_new  = mu1 + cov12 @ inv(cov22) @ (a - mu2)
    cov_new = cov11 - cov12 @ inv(cov22) @ cov21
    return(mu_new, cov_new)

cond_dist([2], x)[0]

array([0.   , 0.   , 0.999])